In [2]:
%pip install pystac_client leafmap rasterio localtileserver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.9/632.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.7/207.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.

In [3]:
from pystac_client import Client
API_ROOT = "https://data.apps.fao.org/geospatial/search/stac"
client = Client.open(API_ROOT)
cols = list(client.get_collections())
print(f"Found {len(cols)} collections")
for c in cols:
    print(f"- {c.id} :: {c.title or ''}")

Found 32 collections
- ASI-A :: ASIS: Agricultural Stress Index (Global, 1 km, Annual)
- ASI-D :: ASIS: Agricultural Stress Index (Global, 1 km,  Dekadal)
- CAM7-SIMPLE-2020 :: Cropland Disagreement of Seven Global Cropland Products
- CG-CA :: Cropgrids Crop Area
- CG-HA :: Cropgrids Harvested Area
- DI-A :: ASIS: Drought Intensity (Global, 1 km, Annual)
- DI-D :: ASIS: Drought Intensity (Global, 1 km, Dekadal)
- GET :: Global daily 1-km actual evapotranspiration from 2000 to 2021
- HDF :: ASIS: Historic Drought Frequency (Global, 1 km, Multi-annual)
- L1-AETI-M :: Actual EvapoTranspiration and Interception (Global - Monthly - 300m)
- L1-PCP-M :: Precipitation (Global - Montly - Approximately 5km)
- L1-QUAL-LST-D :: Quality land surface temperature (Global - Dekadal - 300m)
- L1-RET-A :: Reference Evapotranspiration (Global - Annual - Approximately 30km)
- L1-RET-D :: Reference Evapotranspiration (Global - Dekadal - Approximately 30km)
- L1-UTM-AETI-D :: Actual EvapoTranspiration and I

In [4]:
from datetime import datetime

# === EDIT THESE ===

# You can put: []  or ""  or None  to search without a collection filter.
# Or provide one or many collection ids, e.g. ["L1-UTM-NPP-D"] or "L1-UTM-NPP-D"
COLLECTIONS = "THU-GCI-INTILE"   # or [] or None

# Bounding box in WGS84 [minx, miny, maxx, maxy]; set to None to ignore
BBOX =[118, 38, 122, 40]   # or None

# Time range; set to None to ignore (STAC interval string "start/end")
DATETIME = "2017-01-01/2018-12-31"  # or None

# Optional: LIKE pattern for item id (SQL wildcards: % = any string, _ = single char)
# Example: "%60W.2024-12-D3%"  |  set to None to skip LIKE filtering
ITEM_ID_LIKE = None

# Optional extra STAC 'query' filter
QUERY = {
    # "eo:cloud_cover": {"lt": 20}
}

# Optional cap on results
LIMIT = 100

print("STAC_API:", API_ROOT)
print("COLLECTIONS:", COLLECTIONS)
print("BBOX:", BBOX)
print("DATETIME:", DATETIME)
print("ITEM_ID_LIKE:", ITEM_ID_LIKE)


STAC_API: https://data.apps.fao.org/geospatial/search/stac
COLLECTIONS: THU-GCI-INTILE
BBOX: [118, 38, 122, 40]
DATETIME: 2017-01-01/2018-12-31
ITEM_ID_LIKE: None


In [5]:
from pystac_client import Client

def _normalize_collections(val):
    if val is None:
        return None
    if isinstance(val, str):
        val = val.strip()
        return [val] if val else None
    if isinstance(val, (list, tuple)):
        vals = [v for v in (x.strip() for x in val) if v]
        return vals or None
    return None

client = Client.open(API_ROOT)

search_kwargs = dict(
    collections=_normalize_collections(COLLECTIONS),
    bbox=BBOX or None,
    datetime=DATETIME or None,
    query=QUERY or None,
    limit=LIMIT,
)

# Add LIKE filter only if user provided ITEM_ID_LIKE
if ITEM_ID_LIKE:
    search_kwargs.update({
        "filter_lang": "cql2-json",
        "filter": {
            "op": "like",
            "args": [
                {"property": "id"},
                ITEM_ID_LIKE
            ]
        }
    })

search = client.search(**search_kwargs)

items = list(search.get_items())
print(f"Found {len(items)} items")
if not items:
    raise RuntimeError("No items found. Adjust COLLECTIONS/BBOX/DATETIME/ITEM_ID_LIKE and rerun.")

# Choose an item (first by default)
item = items[0]
print("Selected item:", item.id)


/usr/local/lib/python3.12/dist-packages/pystac_client/item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Found 4 items
Selected item: THU.THU-GCI-INTILE.60E90N.2018


In [6]:
import pandas as pd

def item_summary(it):
    props = it.properties or {}
    return {
        "id": it.id,
        "collection": it.collection_id,
        "datetime": props.get("datetime", None),
        "cloud_cover": props.get("eo:cloud_cover", None),
        "gsd": props.get("gsd", None),
        "assets": ",".join(sorted(it.assets.keys())),
    }

df = pd.DataFrame([item_summary(it) for it in items])
df.head(10)


,id,collection,datetime,cloud_cover,gsd,assets
0,THU.THU-GCI-INTILE.60E90N.2018,THU-GCI-INTILE,None,None,None,"data,preview,thumbnail"
1,THU.THU-GCI-INTILE.60E90N.2017,THU-GCI-INTILE,None,None,None,"data,preview,thumbnail"
2,THU.THU-GCI-INTILE.120E90N.2018,THU-GCI-INTILE,None,None,None,"data,preview,thumbnail"
3,THU.THU-GCI-INTILE.120E90N.2017,THU-GCI-INTILE,None,None,None,"data,preview,thumbnail"


In [7]:
import geopandas as gpd
from shapely.geometry import shape
import leafmap

# Build GeoDataFrame from item geometries
geoms = []
ids = []
cols = []
dts = []

for it in items:
    if it.geometry:
        geoms.append(shape(it.geometry))
        ids.append(it.id)
        cols.append(it.collection_id)
        dts.append(it.properties.get("datetime"))

if not geoms:
    raise RuntimeError("No geometries found in items.")

gdf = gpd.GeoDataFrame(
    {"id": ids, "collection": cols, "datetime": dts},
    geometry=geoms,
    crs="EPSG:4326",
)

# Create map
m = leafmap.Map()
m.add_basemap("CartoDB.Positron")

# Simple, nicer style for footprints
style = {
    "color": "#22c55e",   # outline
    "weight": 2,
    "fillOpacity": 0.1,
}

m.add_gdf(
    gdf,
    layer_name="STAC footprints",
    style=style,
)

# Zoom to all item footprints (correct API usage)
minx, miny, maxx, maxy = gdf.total_bounds
m.zoom_to_bounds([minx, miny, maxx, maxy])

m


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [8]:
import leafmap

def _is_geotiff(a):
    mt = (a.media_type or "").lower()
    href = (a.href or "").lower()
    return ("image/tiff" in mt) or href.endswith((".tif", ".tiff"))

def pick_asset(it):
    # Prefer common keys, fall back to any GeoTIFF
    preferred = ("data", "visual", "rendered_preview", "B04", "B03", "B02", "nir")
    for k in preferred:
        if k in it.assets and _is_geotiff(it.assets[k]):
            return it.assets[k], k
    for k, a in it.assets.items():
        if _is_geotiff(a):
            return a, k
    return None, None

asset, asset_key = pick_asset(item)
if not asset:
    raise RuntimeError(f"No GeoTIFF/COG asset found for item {item.id}")

href = asset.href or asset.get_absolute_href()
if not href:
    raise RuntimeError(f"Asset '{asset_key}' has no usable href.")

print("Rendering item:", item.id)
print("Using asset:", asset_key)
print("URL:", href)

m2 = leafmap.Map()
m2.add_basemap("CartoDB.Positron")

m2.add_raster(
    href,
    layer_name=f"{item.collection_id}:{asset_key}",
    colormap="viridis",   # nicer than raw black/white
    zoom_to_layer=True,
)

m2


Rendering item: THU.THU-GCI-INTILE.60E90N.2018
Using asset: data
URL: https://storage.googleapis.com/fao-gismgr-thu-data/DATA/THU/MOSAICSET/THU-GCI-INTILE/THU.THU-GCI-INTILE.60E90N.2018.tif


Map(center=[60.00072349999999, 89.9999625], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…